In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc","all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,annual_inc,open_acc,total_acc,total_pymnt,loan_status
0,35000.0,0.1308,125890.0,14.0,29.0,35016.528333,Fully Paid
1,20000.0,0.2250,75000.0,8.0,11.0,719.000000,low_risk
2,10500.0,0.1719,66000.0,8.0,14.0,355.290000,low_risk
3,25000.0,0.2000,105000.0,17.0,34.0,873.530000,low_risk
4,20000.0,0.2000,56000.0,8.0,21.0,485.440000,low_risk


# Split the Data into Training and Testing

In [6]:
# Create our features
X = df.drop(columns="loan_status")
X = pd.get_dummies(X)
# Create our target
y = df["loan_status"]

In [7]:
X.describe()

,loan_amnt,int_rate,annual_inc,open_acc,total_acc,total_pymnt
count,96840.000000,96840.000000,9.684000e+04,96840.000000,96840.000000,96840.000000
mean,16642.454822,0.126793,8.516747e+04,11.707239,23.079864,1272.141466
std,10365.336053,0.048470,1.133767e+05,5.978739,12.186639,2796.006935
min,1000.000000,0.060000,0.000000e+00,0.000000,2.000000,0.000000
25%,9000.000000,0.081900,4.900000e+04,7.000000,14.000000,408.060000
50%,15000.000000,0.118000,7.000000e+04,11.000000,21.000000,726.530000
75%,24000.000000,0.155700,1.000000e+05,15.000000,30.000000,1258.670000
max,40000.000000,0.308900,9.000000e+06,72.000000,164.000000,41418.981165


In [8]:
# Check the balance of our target values
y.value_counts()

low_risk       94116
Fully Paid      2157
high_risk        526
Charged Off       41
Name: loan_status, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({'low_risk': 70569,
         'Fully Paid': 1644,
         'high_risk': 385,
         'Charged Off': 32})

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [10]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf = RandomForestClassifier(random_state=1)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [11]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = clf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5988005946549106

In [12]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[    1,     0,     6,     2],
       [    0,   508,     0,     5],
       [    4,     0,    42,    95],
       [    1,    71,    23, 23452]], dtype=int64)

In [13]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       0.17      0.11      1.00      0.13      0.33      0.10         9
 Fully Paid       0.88      0.99      1.00      0.93      0.99      0.99       513
  high_risk       0.59      0.30      1.00      0.40      0.55      0.28       141
   low_risk       1.00      1.00      0.85      1.00      0.92      0.86     23547

avg / total       0.99      0.99      0.85      0.99      0.92      0.85     24210



In [14]:
# List the features sorted in descending order by feature importance
importances = clf.feature_importances_

sorted(zip(clf.feature_importances_, X.columns), reverse=True)

# NEED TO CHANGE HOW THIS IS PRINTED: going to print as an array rn with column name second

[(0.750787394317959, 'total_pymnt'),
 (0.07753788128581024, 'loan_amnt'),
 (0.05711815540737169, 'annual_inc'),
 (0.04272424276437888, 'total_acc'),
 (0.03696139300925079, 'int_rate'),
 (0.03487093321522939, 'open_acc')]

### Easy Ensemble AdaBoost Classifier

In [15]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [16]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6856888437880988

In [17]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[    5,     0,     4,     0],
       [    0,   503,     0,    10],
       [   20,     2,    61,    58],
       [   27,   720,  4573, 18227]], dtype=int64)

In [18]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       0.10      0.56      1.00      0.16      0.74      0.53         9
 Fully Paid       0.41      0.98      0.97      0.58      0.98      0.95       513
  high_risk       0.01      0.43      0.81      0.03      0.59      0.34       141
   low_risk       1.00      0.77      0.90      0.87      0.83      0.69     23547

avg / total       0.98      0.78      0.90      0.86      0.84      0.69     24210

